#### Info
Este notebook explica los conceptos y el proceso de reservas RPND y PCR-CP que se busca replicar con base en las definiciones 
realizadas por los equipos de actuaría y de gestión contable del proyecto feniX IFRS17. Los datos utilizados son de muestra y no reflejan movimientos ni nombres/códigos reales en los sistemas de información, las columnas se nombraron de forma descriptiva y deben conectarse con las definiciones en las HU de entradas.

In [1]:
import src.parametros as p
import src.aux_tools as aux_tools
import src.prep_insumo as prep_data
import src.devenga as devg
import src.fluctuacion as fluc
import src.mapeo_contable as mapcont
import polars as pl


# lee los parámetros e insumos relevantes
FECHA_VALORACION = p.FECHA_VALORACION
FECHA_VALORACION_ANTERIOR = p.FECHA_VALORACION_ANTERIOR
FECHA_TRANSICION = p.FECHA_TRANSICION
# Insumos transversales
param_contab = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_PARAMETROS_CONTAB).filter(
    pl.col('estado_insumo') == 1)   # Solo se usan las configuraciones activas (1)
excepciones = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_EXCEPCIONES_50_50)
gasto = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_GASTO)
tasa_cambio = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_MONEDA)
descuentos = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_DESCUENTO)
# El diccionario o tabla de correspondencia de outputs con entradas contables
input_map_bts = pl.read_excel(p.RUTA_REL_BT)
input_tipo_seguro = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_TIPO_SEGURO)
# Insumos de cuenta corriente y cartera
# input_cuentas_reaseguro = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_CUENTAS_REA)
# Insumo de riesgo de credito reaseguradores
input_riesgo_credito = pl.read_excel(p.RUTA_RIESGO_CREDITO)

# Insumos de recibos contabilizados en SAP
produccion_dir = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_PDN)
cesion_rea = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_CESION)
comision_rea = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_COMISION_REA)
costo_contrato_rea = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_COSTO_CONTRATO)
seguimiento_rea = pl.read_excel(p.RUTA_INSUMOS, sheet_name=p.HOJA_SEGUIMIENTO_REA)

.when([(col("nivel_detalle")) == ("recibo")]).then(col("fecha_inicio_vigencia_recibo")).otherwise(.when([(col("nivel_detalle")) == ("cobertura")]).then(col("fecha_inicio_vigencia_cobertura")).otherwise(null))


# PCR CP - Provisión de Cobertura Remanente Corto Plazo

La provisión de cobertura remanente se define bajo IFRS17 como una reserva que representa el valor correspondiente al riesgo futuro no corrido bajo un contrato de seguros.
La reserva puede representar un pasivo (una obligación de la aseguradora) o un activo (un derecho de la aseguradora) dependiendo del tipo de contrato.

Se puede entender como el valor de la prima del seguro que debe tenerse guardado por parte del asegurador en cada momento para responder por siniestros que aún no han ocurrido y, en caso de que el contrato se cancele durante la vigencia, este valor se le debe devolver al tomador del seguro.

En corto plazo (contratos con duración de hasta 1 año), bajo IFRS17 se permite la simplificación de la reserva al esquema de medición de Asignación de Primas (PAA), es decir, se toma la prima del contrato y se reserva desde su emisión hasta el fin de vigencia del contrato, asignando a cada periodo contable (mes) la porción correspondiente de la prima según el riesgo transcurrido en ése periodo. Este esquema, es equivalente al utilizado para medir la reserva de prima no devengada RPND bajo IFRS4.

## Saldo de Reserva:
El saldo es el valor de la reserva en el momento de medición o valoración t.
$$ \text{Saldo} = \text{Prima} \times \%\text{Riesgo no corrido}_t $$

## Movimientos de Reserva:
La reserva debe ser suficiente y ajustada al riesgo futuro en cada momento de medición o valoración t, por lo tanto, mensualmente, en cada cierre se realizan movimientos de constitución (incrementar reserva) o de liberación (reducir reserva) para ajustarla al momento de la medición. 

- Los signos de los movimientos de constitución son opuestos a los de los movimientos de liberación para un mismo componente.
- La constitución se realiza solo en el momento en que se emite la prima y es el nacimiento de la reserva
- La liberación se hace en cada momento t mientras la póliza esté vigente:

$$ \text{liberacion} = \text{Prima} \times \%\text{Riesgo corrido en el mes}_t $$


# 1. Preparación de Insumos para el Devengamiento

Los insumos de la PCR pueden variar porque tiene varios componentes internos que, por definición metodológica, se requerirán por separado para adapatarse a los diferentes estándares contables y poderlos agregar o desagregar. Sin embargo, el cálculo de devengamiento tiene una lógica común para todos los componentes, con algunas excepciones de reglas, omisiones de cálculo o nivel de detalle que se consideran parametrizables. 

Por lo tanto, el objetivo de la preparación de insumos es aplicar los pasos a los datos de entrada para asignarle a cada entrada las variables que recibe el devengamiento (fecha_constitucion, fecha_inicio_devengo, fecha_fin_devengo, valor_base_devengo) y el tipo de método (diario o regla del 50%).

Cada tipo de insumo que define un componente relevante para la PCR tiene una función de preparación para ser explícitos en los pasos aplicables según corresponda.

## 1.1. PCR del Seguro Directo
La PCR del seguro directo representa una reserva para la aseguradora, pues debo guardar la parte de la prima correspondiente al servicio que todavía no se ha prestado. Tiene varios componentes internos que deben medirse por separado según el estándar de contabilidad y afecta según el signo (-) incrementa el saldo de reserva y (+) disminuye el saldo de reserva:

- PCR de prima emitida (-)
- PCR del gasto de adquisición por comisiones (+)
- PCR del gasto de adquisición por otros conceptos (+)
- PCR del descuento comercial (-)

### Componente de Prima Emitida

In [2]:
# prepara el insumo para el devengo
input_prod_dir = prep_data.prep_input_prima_directo(produccion_dir, param_contab, excepciones, FECHA_VALORACION)
input_prod_dir

aux_casuistica,tipo_insumo,tipo_negocio,poliza,fecha_expedicion_poliza,recibo,amparo,poliza_certificado,compania,ramo_sura,canal,producto,tipo_op,moneda,fecha_contabilizacion_recibo,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,valor_prima_emitida,tipo_contabilidad,componente,tipo_contrato,nivel_detalle,signo_constitucion,candidato_devengo_50_50,fecha_inicio_vigencia,fecha_constitucion,fecha_inicio_devengo,fecha_fin_devengo,valor_base_devengo
str,str,str,i64,date,i64,str,i64,str,str,str,str,str,str,date,date,date,date,date,f64,str,str,str,str,i64,i32,date,date,date,date,f64
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""ifrs17_local""","""prima""","""directo""","""cobertura""",-1,0,2024-01-30,2024-01-30,2024-01-30,2024-02-15,50.0
"""mensual_mes_fin""","""produccion_directo""","""directo""",345675387,2024-02-20,2938431543,"""HURTO""",345675807,"""01""","""040""","""sucursal""","""plan_2""","""emision""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100.0,"""ifrs17_local""","""prima""","""directo""","""cobertura""",-1,0,2024-01-15,2024-02-20,2024-02-20,2024-02-15,100.0
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""ifrs4""","""prima""","""directo""","""recibo""",-1,0,2024-01-30,2024-01-30,2024-01-30,2024-02-15,50.0
"""mensual_mes_fin""","""produccion_directo""","""directo""",345675387,2024-02-20,2938431543,"""HURTO""",345675807,"""01""","""040""","""sucursal""","""plan_2""","""emision""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100.0,"""ifrs4""","""prima""","""directo""","""recibo""",-1,1,2024-01-15,2024-02-20,2024-02-20,2024-02-15,100.0
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""ifrs17_corporativo""","""prima""","""directo""","""cobertura""",-1,0,2024-01-30,2024-01-30,2024-01-30,2024-02-15,50.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""terremoto""","""produccion_directo""","""directo""",130534534,2024-01-30,5340312414,"""TERREMOTO""",425544224,"""01""","""130""","""sucursal""","""plan_1""","""emision""","""COP""",2024-01-30,2024-01-30,2025-01-30,2024-01-30,2025-01-30,1200.0,"""ifrs4""","""prima""","""directo""","""recibo""",-1,1,2024-01-30,2024-01-30,2024-01-30,2025-01-30,1200.0
"""anual_pre_vigencia""","""produccion_directo""","""directo""",763894523,2024-01-15,2938433603,"""HOSPITALARIO""",763894943,"""02""","""091""","""sucursal""","""plan_3""","""emision""","""COP""",2024-01-15,2024-01-16,2025-01-15,2024-01-16,2025-01-15,1000.0,"""ifrs17_corporativo""","""prima""","""directo""","""cobertura""",-1,0,2024-01-16,2024-01-15,2024-01-16,2025-01-15,1000.0
"""modificacion_valorable""","""produccion_directo""","""directo""",763894523,2024-01-30,2938433603,"""HOSPITALARIO""",763894943,"""02""","""091""","""sucursal""","""plan_3""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,200.0,"""ifrs17_corporativo""","""prima""","""directo""","""cobertura""",-1,0,2024-01-30,2024-01-30,2024-01-30,2024-02-15,200.0


### Componente de Descuento Comercial
- El valor a devengar se obtiene reconstruyendo la prima original antes de cualquier descuento y después aplicando el descuento comercial sobre la prima reconstruida
$$Prima Reconstruida = Prima Emitida / (Porc dcto Tecnico + Porc dcto Comercial)$$
$$DescuentoComercial = PrimaReconstruida * PorcdctoComercial$$

In [3]:
# prepara el insumo para el devengo
input_dcto_dir = prep_data.prep_input_dcto_directo(produccion_dir, param_contab, excepciones, descuentos, FECHA_VALORACION)
input_dcto_dir

aux_casuistica,tipo_insumo,tipo_negocio,poliza,fecha_expedicion_poliza,recibo,amparo,poliza_certificado,compania,ramo_sura,canal,producto,tipo_op,moneda,fecha_contabilizacion_recibo,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,valor_prima_emitida,tipo_contabilidad,componente,tipo_contrato,nivel_detalle,signo_constitucion,candidato_devengo_50_50,podto_comercial,podto_tecnico,fecha_inicio_vigencia,fecha_constitucion,fecha_inicio_devengo,fecha_fin_devengo,valor_base_devengo
str,str,str,i64,date,i64,str,i64,str,str,str,str,str,str,date,date,date,date,date,f64,str,str,str,str,i64,i32,f64,f64,date,date,date,date,f64
"""garantia_extendida_pre_riesgo""","""dcto_directo""","""directo""",259823906,2023-07-15,2940977660,"""DANOS""",239524652,"""01""","""009""","""sucursal""","""plan_3""","""emision""","""COP""",2023-07-15,2023-01-15,2025-01-15,2023-01-15,2025-01-15,1200.0,"""ifrs4""","""descuento_comercial""","""directo""","""recibo""",-1,1,0.05,0.05,2023-01-15,2023-07-15,2023-07-15,2025-01-15,66.666667
"""garantia_extendida_en_riesgo""","""dcto_directo""","""directo""",920350354,2024-07-01,9024605326,"""DANOS""",920583545,"""01""","""009""","""sucursal""","""plan_2""","""emision""","""COP""",2024-07-01,2023-01-15,2025-01-15,2023-01-15,2025-01-15,1200.0,"""ifrs4""","""descuento_comercial""","""directo""","""recibo""",-1,1,0.03,0.08,2023-01-15,2024-07-01,2024-07-01,2025-01-15,40.449438
"""anual_pre_vigencia""","""dcto_directo""","""directo""",124324092,2023-11-15,2938430307,"""INVALIDEZ""",124324512,"""02""","""083""","""banca""","""plan_3""","""emision""","""COP""",2023-11-15,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200.0,"""ifrs4""","""descuento_comercial""","""directo""","""recibo""",-1,1,0.08,0.01,2024-01-15,2023-11-15,2024-01-15,2025-01-15,105.494505
"""anual_misma_vigencia""","""dcto_directo""","""directo""",124324091,2024-01-01,2938429895,"""INVALIDEZ""",124324511,"""02""","""083""","""banca""","""plan_3""","""emision""","""USD""",2024-01-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200.0,"""ifrs4""","""descuento_comercial""","""directo""","""recibo""",-1,1,0.06,0.04,2024-01-15,2024-01-01,2024-01-15,2025-01-15,80.0
"""mensual_pre_vigencia""","""dcto_directo""","""directo""",634535346,2023-12-15,2938431131,"""AMBULATORIO""",634535766,"""02""","""091""","""sucursal""","""plan_3""","""emision""","""COP""",2023-12-15,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100.0,"""ifrs4""","""descuento_comercial""","""directo""","""recibo""",-1,1,0.03,0.06,2024-01-15,2023-12-15,2024-01-15,2024-02-15,3.296703
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""anual_en_vigencia""","""dcto_directo""","""directo""",421093128,2024-04-01,2938429483,"""VIDA""",421093548,"""02""","""083""","""banca""","""plan_1""","""emision""","""COP""",2024-04-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200.0,"""ifrs4""","""descuento_comercial""","""directo""","""recibo""",-1,1,0.04,0.03,2024-01-15,2024-04-01,2024-04-01,2025-01-15,51.612903
"""anual_en_vigencia""","""dcto_directo""","""directo""",421093128,2024-04-01,2938432779,"""VIDA""",421093548,"""02""","""083""","""banca""","""plan_1""","""cancelacion""","""COP""",2024-07-30,2024-07-30,2025-01-15,2024-07-30,2025-01-15,-555.858311,"""ifrs4""","""descuento_comercial""","""directo""","""recibo""",-1,1,0.04,0.03,2024-07-30,2024-07-30,2024-07-30,2025-01-15,-23.907884
"""terremoto""","""dcto_directo""","""directo""",130534534,2024-01-30,5340312414,"""TERREMOTO""",425544224,"""01""","""130""","""sucursal""","""plan_1""","""emision""","""COP""",2024-01-30,2024-01-30,2025-01-30,2024-01-30,2025-01-30,1200.0,"""ifrs4""","""descuento_comercial""","""directo""","""recibo""",-1,1,0.0,0.0,2024-01-30,2024-01-30,2024-01-30,2025-01-30,0.0


### Componente Gasto de Expedición
- Separado entre gasto de expedición por comisiones y otros, es decir que entra una linea por cada tipo de gasto, para esto, la función de preparación hace un melt (wide to long)
- Se pueden parametrizar porcentajes distintos de gasto de expedición por tipo de contabilidad. Por ejemplo, el corporativo puede tener un porcentaje distinto al local debido a diferencias en los criterios de clasificación del gasto.
- Es importante revisar que los cruces garanticen que no se generen duplicados porque se usa el comodín "*" cuando no existe detalle para el canal y/o producto y por lo tanto aplica para todo un ramo.

In [4]:
# prepara el insumo para el devengo
input_gasto_dir = prep_data.prep_input_gasto_directo(produccion_dir, param_contab, excepciones, gasto, FECHA_VALORACION)
input_gasto_dir

aux_casuistica,tipo_insumo,tipo_negocio,poliza,fecha_expedicion_poliza,recibo,amparo,poliza_certificado,compania,ramo_sura,canal,producto,tipo_op,moneda,fecha_contabilizacion_recibo,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,valor_prima_emitida,tipo_gasto,porc_gasto,prioridad_match,rn,tipo_contabilidad,componente,tipo_contrato,nivel_detalle,signo_constitucion,candidato_devengo_50_50,fecha_inicio_vigencia,fecha_constitucion,fecha_inicio_devengo,fecha_fin_devengo,valor_base_devengo
str,str,str,i64,date,i64,str,i64,str,str,str,str,str,str,date,date,date,date,date,f64,str,f64,i32,i64,str,str,str,str,i64,i32,date,date,date,date,f64
"""anual_en_vigencia""","""gasto_otro_directo""","""directo""",421093128,2024-04-01,2938429483,"""VIDA""",421093548,"""02""","""083""","""banca""","""plan_1""","""emision""","""COP""",2024-04-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200.0,"""expedicion_otros""",0.6,2,1,"""ifrs17_corporativo""","""gasto_exp_otros""","""directo""","""cobertura""",1,0,2024-01-15,2024-04-01,2024-04-01,2025-01-15,720.0
"""mensual_mes_fin""","""gasto_otro_directo""","""directo""",345675387,2024-02-20,2938431543,"""HURTO""",345675807,"""01""","""040""","""sucursal""","""plan_2""","""emision""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100.0,"""expedicion_otros""",0.5,2,1,"""ifrs4""","""gasto_exp_otros""","""directo""","""recibo""",1,1,2024-01-15,2024-02-20,2024-02-20,2024-02-15,50.0
"""mensual_post_vigencia""","""gasto_otro_directo""","""directo""",436736799,2024-03-15,2938431955,"""RC""",436737219,"""01""","""040""","""sucursal""","""plan_2""","""emision""","""COP""",2024-03-15,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100.0,"""expedicion_otros""",0.5,2,1,"""ifrs17_corporativo""","""gasto_exp_otros""","""directo""","""cobertura""",1,0,2024-01-15,2024-03-15,2024-03-15,2024-02-15,50.0
"""terremoto""","""gasto_otro_directo""","""directo""",130534534,2024-01-30,5340312414,"""TERREMOTO""",425544224,"""01""","""130""","""sucursal""","""plan_1""","""emision""","""COP""",2024-01-30,2024-01-30,2025-01-30,2024-01-30,2025-01-30,1200.0,"""expedicion_otros""",0.04,3,1,"""ifrs17_corporativo""","""gasto_exp_otros""","""directo""","""cobertura""",1,0,2024-01-30,2024-01-30,2024-01-30,2025-01-30,48.0
"""garantia_extendida_pre_riesgo""","""gasto_otro_directo""","""directo""",259823906,2023-07-15,2940977660,"""DANOS""",239524652,"""01""","""009""","""sucursal""","""plan_3""","""emision""","""COP""",2023-07-15,2023-01-15,2025-01-15,2023-01-15,2025-01-15,1200.0,"""expedicion_otros""",0.3,2,1,"""ifrs4""","""gasto_exp_otros""","""directo""","""recibo""",1,1,2023-01-15,2023-07-15,2023-07-15,2025-01-15,360.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""anual_en_vigencia""","""gasto_otro_directo""","""directo""",421093128,2024-04-01,2938429483,"""VIDA""",421093548,"""02""","""083""","""banca""","""plan_1""","""emision""","""COP""",2024-04-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200.0,"""expedicion_otros""",0.6,2,1,"""ifrs4""","""gasto_exp_otros""","""directo""","""recibo""",1,1,2024-01-15,2024-04-01,2024-04-01,2025-01-15,720.0
"""terremoto""","""gasto_otro_directo""","""directo""",130534534,2024-01-30,5340312414,"""TERREMOTO""",425544224,"""01""","""130""","""sucursal""","""plan_1""","""emision""","""COP""",2024-01-30,2024-01-30,2025-01-30,2024-01-30,2025-01-30,1200.0,"""expedicion_otros""",0.04,3,1,"""ifrs4""","""gasto_exp_otros""","""directo""","""recibo""",1,1,2024-01-30,2024-01-30,2024-01-30,2025-01-30,48.0
"""garantia_extendida_pre_riesgo""","""gasto_otro_directo""","""directo""",259823906,2023-07-15,2940977660,"""DANOS""",239524652,"""01""","""009""","""sucursal""","""plan_3""","""emision""","""COP""",2023-07-15,2023-01-15,2025-01-15,2023-01-15,2025-01-15,1200.0,"""expedicion_otros""",0.3,2,1,"""ifrs17_corporativo""","""gasto_exp_otros""","""directo""","""cobertura""",1,0,2023-01-15,2023-07-15,2023-07-15,2025

## 1.2. PCR del Reaseguro Proporcional
La PCR del reaseguro representa un activo para la aseguradora, pues debo reconocer que tengo derecho a recobrar la parte de la prima cedida correspondiente al servicio que todavía no se ha prestado por el reasegurador. Tiene varios componentes internos que deben medirse por separado según el estándar de contabilidad y afecta según el signo (+) incrementa el saldo del activo y (-) disminuye el saldo del activo:

- PCR de prima cedida (+)
- PCR de las comisiones de reaseguro (-)
- PCR del descuento comercial de la prima cedida (+)
- PCR del costo de contrato (+)
- PCR del gasto de adquisición por comisiones (-) solo IFRS4
- PCR del gasto de adquisición por otros conceptos (-) solo IFRS4
- Cuenta por pagar primas cedidas (-) solo IFRS17 Corp
- Cuenta por cobrar comisiones reaseguro (+) solo IFRS17 Corp

### Componente de Prima Cedida

In [5]:
# prepara el insumo para el devengo
input_cesion_rea = prep_data.prep_input_prima_rea(cesion_rea, param_contab, excepciones, FECHA_VALORACION)
input_cesion_rea

aux_casuistica,tipo_insumo,contrato_reaseguro,tipo_negocio,nit_reasegurador,tipo_reasegurador,porc_participacion_reasegurador,poliza,fecha_expedicion_poliza,recibo,fe_ini_vig_contrato_reaseguro,fe_fin_vig_contrato_reaseguro,amparo,poliza_certificado,compania,ramo_sura,canal,producto,tipo_op,moneda,fecha_contabilizacion_recibo,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,valor_prima_emitida,valor_prima_cedida,porc_cesion,tipo_contabilidad,componente,tipo_contrato,nivel_detalle,signo_constitucion,candidato_devengo_50_50,fecha_inicio_vigencia,fecha_constitucion,fecha_inicio_devengo,fecha_fin_devengo,valor_base_devengo
str,str,i64,str,i64,str,f64,i64,date,i64,date,date,str,i64,str,str,str,str,str,str,date,date,date,date,date,i64,i64,f64,str,str,str,str,i64,i32,date,date,date,date,f64
"""anual_misma_vigencia""","""cesion_rea_prop""",1111,"""mantenido""",3693358,"""ext""",0.3,124324091,2024-01-01,293842989501,2023-07-01,2024-06-30,"""INVALIDEZ""",124324511,"""02""","""083""","""banca""","""plan_3""","""emision""","""USD""",2024-01-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200,600,0.5,"""ifrs4""","""prima""","""reaseguro_proporcional""","""recibo""",1,1,2024-01-15,2024-01-01,2024-01-15,2025-01-15,180.0
"""anual_pre_vigencia""","""cesion_rea_prop""",1111,"""mantenido""",2374188,"""int""",0.4,124324092,2023-11-15,293843030701,2023-07-01,2024-06-30,"""INVALIDEZ""",124324512,"""02""","""083""","""banca""","""plan_3""","""emision""","""COP""",2023-11-15,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200,600,0.5,"""ifrs4""","""prima""","""reaseguro_proporcional""","""recibo""",1,1,2024-01-15,2023-11-15,2024-01-15,2025-01-15,240.0
"""anual_misma_vigencia""","""cesion_rea_prop""",1111,"""mantenido""",3693358,"""ext""",0.3,124324091,2024-01-01,293842989501,2023-07-01,2024-06-30,"""INVALIDEZ""",124324511,"""02""","""083""","""banca""","""plan_3""","""emision""","""USD""",2024-01-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200,600,0.5,"""ifrs17_corporativo""","""prima""","""reaseguro_proporcional""","""cobertura""",1,0,2024-01-15,2024-01-01,2024-01-15,2025-01-15,180.0
"""anual_pre_vigencia""","""cesion_rea_prop""",1111,"""mantenido""",2374188,"""int""",0.4,124324092,2023-11-15,293843030701,2023-07-01,2024-06-30,"""INVALIDEZ""",124324512,"""02""","""083""","""banca""","""plan_3""","""emision""","""COP""",2023-11-15,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200,600,0.5,"""ifrs17_corporativo""","""prima""","""reaseguro_proporcional""","""cobertura""",1,0,2024-01-15,2023-11-15,2024-01-15,2025-01-15,240.0
"""anual_misma_vigencia""","""cesion_rea_prop""",1111,"""mantenido""",3693358,"""ext""",0.3,124324091,2024-01-01,293842989501,2023-07-01,2024-06-30,"""INVALIDEZ""",124324511,"""02""","""083""","""banca""","""plan_3""","""emision""","""USD""",2024-01-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200,600,0.5,"""ifrs17_local""","""prima""","""reaseguro_proporcional""","""cobertura""",1,0,2024-01-15,2024-01-01,2024-01-15,2025-01-15,180.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""mensual_mes_fin""","""cesion_rea_prop""",1245,"""mantenido""",3693358,"""ext""",0.5,345675387,2024-02-20,293843154301,2024-01-01,2024-12-31,"""HURTO""",345675807,"""01""","""040""","""sucursal""","""plan_2""","""emision""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100,60,0.6,"""ifrs4""","""prima""","""reaseguro_proporcional""","""recibo""",1,1,2024-01-15,2024-02-20,2024-02-20,2024-02-15,30.0
"""mensual_pre_vigencia""","""cesion_rea_prop""",1113,"""mantenido""",3339928,"""ext""",1.0,634535346,2023-12-15,293843113101,2023-07-01,2024-06-30,"""AMBULATORIO""",634535766,"""02""","""091""","""sucursal""","""plan_3""","""emision""","""COP""",2023-12-15,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100,30,0.3,"""ifrs17_corporativo""","""prima""","""reaseguro_proporcional""","""cobertura""",1,0,2024-01-15,2023-12-15,2024-01-15,2024-02-15,30.0
"""

### Componente de descuento comercial sobre la cesión
- En linea con la circular externa 2255 de 2015, el descuento comercial nos lleva a reconocer una reserva más alta del seguro directo, esto se traduce en una reserva más baja por este componente de descuento comercial en el reaseguro proporcional
- Al igual que en el descuento para el seguro directo, se reconstruye la prima completa antes de descuento, pero no emitida sino cedida

$$Prima Cedida Reconstruida = Prima Cedida / (Porc dcto Tecnico + Porc dcto Comercial)$$
$$DescuentoComercialCedido = PrimaCedidaReconstruida * PorcdctoComercial$$

In [6]:
# prepara el insumo para el devengo
input_dcto_rea = prep_data.prep_input_dcto_rea(cesion_rea, param_contab, excepciones, descuentos, FECHA_VALORACION)
input_dcto_rea

aux_casuistica,tipo_insumo,contrato_reaseguro,tipo_negocio,nit_reasegurador,tipo_reasegurador,porc_participacion_reasegurador,poliza,fecha_expedicion_poliza,recibo,fe_ini_vig_contrato_reaseguro,fe_fin_vig_contrato_reaseguro,amparo,poliza_certificado,compania,ramo_sura,canal,producto,tipo_op,moneda,fecha_contabilizacion_recibo,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,valor_prima_emitida,valor_prima_cedida,porc_cesion,tipo_contabilidad,componente,tipo_contrato,nivel_detalle,signo_constitucion,candidato_devengo_50_50,podto_comercial,podto_tecnico,fecha_inicio_vigencia,fecha_constitucion,fecha_inicio_devengo,fecha_fin_devengo,valor_base_devengo
str,str,i64,str,i64,str,f64,i64,date,i64,date,date,str,i64,str,str,str,str,str,str,date,date,date,date,date,i64,i64,f64,str,str,str,str,i64,i32,f64,f64,date,date,date,date,f64
"""mensual_mes_fin""","""dcto_rea_prop""",1245,"""mantenido""",2374188,"""int""",0.5,345675387,2024-02-20,293843154301,2024-01-01,2024-12-31,"""HURTO""",345675807,"""01""","""040""","""sucursal""","""plan_2""","""emision""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100,60,0.6,"""ifrs4""","""descuento_comercial""","""reaseguro_proporcional""","""recibo""",-1,1,0.08,0.04,2024-01-15,2024-02-20,2024-02-20,2024-02-15,2.727273
"""anual_misma_vigencia""","""dcto_rea_prop""",1111,"""mantenido""",3693358,"""ext""",0.3,124324091,2024-01-01,293842989501,2023-07-01,2024-06-30,"""INVALIDEZ""",124324511,"""02""","""083""","""banca""","""plan_3""","""emision""","""USD""",2024-01-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200,600,0.5,"""ifrs4""","""descuento_comercial""","""reaseguro_proporcional""","""recibo""",-1,1,0.06,0.04,2024-01-15,2024-01-01,2024-01-15,2025-01-15,12.0
"""anual_pre_vigencia""","""dcto_rea_prop""",1111,"""mantenido""",2374188,"""int""",0.4,124324092,2023-11-15,293843030701,2023-07-01,2024-06-30,"""INVALIDEZ""",124324512,"""02""","""083""","""banca""","""plan_3""","""emision""","""COP""",2023-11-15,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200,600,0.5,"""ifrs4""","""descuento_comercial""","""reaseguro_proporcional""","""recibo""",-1,1,0.08,0.01,2024-01-15,2023-11-15,2024-01-15,2025-01-15,21.098901
"""mensual_pre_vigencia""","""dcto_rea_prop""",1113,"""mantenido""",3339928,"""ext""",1.0,634535346,2023-12-15,293843113101,2023-07-01,2024-06-30,"""AMBULATORIO""",634535766,"""02""","""091""","""sucursal""","""plan_3""","""emision""","""COP""",2023-12-15,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100,30,0.3,"""ifrs4""","""descuento_comercial""","""reaseguro_proporcional""","""recibo""",-1,1,0.03,0.06,2024-01-15,2023-12-15,2024-01-15,2024-02-15,0.989011
"""mensual_mes_fin""","""dcto_rea_prop""",1245,"""mantenido""",3693358,"""ext""",0.5,345675387,2024-02-20,293843154301,2024-01-01,2024-12-31,"""HURTO""",345675807,"""01""","""040""","""sucursal""","""plan_2""","""emision""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100,60,0.6,"""ifrs4""","""descuento_comercial""","""reaseguro_proporcional""","""recibo""",-1,1,0.08,0.04,2024-01-15,2024-02-20,2024-02-20,2024-02-15,2.727273
"""anual_misma_vigencia""","""dcto_rea_prop""",1111,"""mantenido""",2374188,"""int""",0.4,124324091,2024-01-01,293842989501,2023-07-01,2024-06-30,"""INVALIDEZ""",124324511,"""02""","""083""","""banca""","""plan_3""","""emision""","""USD""",2024-01-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200,600,0.5,"""ifrs4""","""descuento_comercial""","""reaseguro_proporcional""","""recibo""",-1,1,0.06,0.04,2024-01-15,2024-01-01,2024-01-15,2025-01-15,16.0
"""anual_misma_vigencia""","""dcto_rea_prop""",1111,"""mantenido""",6987458,"""int""",0.3,124324091,2024-01-01,293842989501,2023-07-01,2024-06-30,"""INVALIDEZ""",124324511,"""02""","""083""","""banca""","""plan_3""","""emision""","""USD""",2024-01-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200,600,0.5,"""ifrs4""","""descuento_comercial""","""reaseguro_proporcional""","""recibo""

### Componente de Gasto de Expedición sobre la Cesión
- Técnicamente no debería netearse de la reserva de reaseguro el gasto de expedición porque no está asociado a la cesión, sino a la emisión del seguro directo. Sin embargo, como hoy se hace por principio de simetría bajo NIIF4, se mantendrá este cálculo y se eliminará el efecto con el ajuste del DAC cedido como se definió en la política contable de la compañía.

- En ifrs17 no aplica, por eso no existe la combinación en el cruce de parametros de contabilidad

In [7]:
# prepara el insumo para el devengo
input_gasto_rea = prep_data.prep_input_gasto_rea(cesion_rea, param_contab, excepciones, gasto, FECHA_VALORACION)
input_gasto_rea

aux_casuistica,tipo_insumo,contrato_reaseguro,tipo_negocio,nit_reasegurador,tipo_reasegurador,porc_participacion_reasegurador,poliza,fecha_expedicion_poliza,recibo,fe_ini_vig_contrato_reaseguro,fe_fin_vig_contrato_reaseguro,amparo,poliza_certificado,compania,ramo_sura,canal,producto,tipo_op,moneda,fecha_contabilizacion_recibo,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,valor_prima_emitida,valor_prima_cedida,porc_cesion,tipo_gasto,porc_gasto,prioridad_match,rn,tipo_contabilidad,componente,tipo_contrato,nivel_detalle,signo_constitucion,candidato_devengo_50_50,fecha_inicio_vigencia,fecha_constitucion,fecha_inicio_devengo,fecha_fin_devengo,valor_base_devengo
str,str,i64,str,i64,str,f64,i64,date,i64,date,date,str,i64,str,str,str,str,str,str,date,date,date,date,date,i64,i64,f64,str,f64,i32,i64,str,str,str,str,i64,i32,date,date,date,date,f64
"""anual_pre_vigencia""","""gasto_comi_rea_prop""",1111,"""mantenido""",6987458,"""int""",0.3,124324092,2023-11-15,293843030701,2023-07-01,2024-06-30,"""INVALIDEZ""",124324512,"""02""","""083""","""banca""","""plan_3""","""emision""","""COP""",2023-11-15,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200,600,0.5,"""expedicion_comisiones""",1.1,2,1,"""ifrs4""","""gasto_exp_comisiones""","""reaseguro_proporcional""","""recibo""",1,1,2024-01-15,2023-11-15,2024-01-15,2025-01-15,198.0
"""mensual_mes_fin""","""gasto_comi_rea_prop""",1245,"""mantenido""",2374188,"""int""",0.5,345675387,2024-02-20,293843154301,2024-01-01,2024-12-31,"""HURTO""",345675807,"""01""","""040""","""sucursal""","""plan_2""","""emision""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100,60,0.6,"""expedicion_comisiones""",0.1,2,1,"""ifrs4""","""gasto_exp_comisiones""","""reaseguro_proporcional""","""recibo""",1,1,2024-01-15,2024-02-20,2024-02-20,2024-02-15,3.0
"""anual_misma_vigencia""","""gasto_comi_rea_prop""",1111,"""mantenido""",2374188,"""int""",0.4,124324091,2024-01-01,293842989501,2023-07-01,2024-06-30,"""INVALIDEZ""",124324511,"""02""","""083""","""banca""","""plan_3""","""emision""","""USD""",2024-01-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,1200,600,0.5,"""expedicion_comisiones""",1.1,2,1,"""ifrs4""","""gasto_exp_comisiones""","""reaseguro_proporcional""","""recibo""",1,1,2024-01-15,2024-01-01,2024-01-15,2025-01-15,264.0
"""mensual_pre_vigencia""","""gasto_comi_rea_prop""",1113,"""mantenido""",3339928,"""ext""",1.0,634535346,2023-12-15,293843113101,2023-07-01,2024-06-30,"""AMBULATORIO""",634535766,"""02""","""091""","""sucursal""","""plan_3""","""emision""","""COP""",2023-12-15,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100,30,0.3,"""expedicion_comisiones""",0.15,2,1,"""ifrs4""","""gasto_exp_comisiones""","""reaseguro_proporcional""","""recibo""",1,1,2024-01-15,2023-12-15,2024-01-15,2024-02-15,4.5


### Componente de Comisiones de Reaseguro

- En IFRS4 no hacía parte de la RPND, se contabiliza aparte, pero se podría implementar para eliminar el devengamiento manual
- En IFRS17 se reconocerá como un flujo asociado al reaseguro, por lo cual hará parte de la PCR

In [8]:
# prepara el insumo para el devengo
input_comi_rea = prep_data.prep_input_comi_rea(comision_rea, param_contab, excepciones, FECHA_VALORACION)
input_comi_rea

aux_casuistica,tipo_insumo,contrato_reaseguro,tipo_negocio,nit_reasegurador,tipo_reasegurador,porc_participacion_reasegurador,recibo,fe_ini_vig_contrato_rea,compania,ramo_sura,poliza,amparo,poliza_certificado,canal_directo,producto,moneda,fecha_contabilizacion_recibo,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,valor_comision_rea,porc_comision_rea,tipo_contabilidad,componente,tipo_contrato,nivel_detalle,signo_constitucion,candidato_devengo_50_50,fecha_inicio_vigencia,fecha_constitucion,fecha_inicio_devengo,fecha_fin_devengo,valor_base_devengo
str,str,i64,str,i64,str,f64,i64,date,str,str,i64,str,i64,str,str,str,date,date,date,date,date,i64,f64,str,str,str,str,i64,i32,date,date,date,date,f64
"""anual_misma_vigencia""","""comision_rea_prop""",1111,"""mantenido""",2374188,"""int""",0.4,6720148806,2023-07-01,"""02""","""083""",124324091,"""INVALIDEZ""",124324511,"""banca""","""plan_3""","""USD""",2024-01-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,120,0.2,"""ifrs17_corporativo""","""comision_cesion""","""reaseguro_proporcional""","""cobertura""",-1,0,2024-01-15,2024-01-01,2024-01-15,2025-01-15,48.0
"""anual_misma_vigencia""","""comision_rea_prop""",1111,"""mantenido""",2374188,"""int""",0.4,6720148806,2023-07-01,"""02""","""083""",124324091,"""INVALIDEZ""",124324511,"""banca""","""plan_3""","""USD""",2024-01-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,120,0.2,"""ifrs17_local""","""comision_cesion""","""reaseguro_proporcional""","""cobertura""",-1,0,2024-01-15,2024-01-01,2024-01-15,2025-01-15,48.0
"""anual_misma_vigencia""","""comision_rea_prop""",1111,"""mantenido""",2374188,"""int""",0.4,6720148806,2023-07-01,"""02""","""083""",124324091,"""INVALIDEZ""",124324511,"""banca""","""plan_3""","""USD""",2024-01-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,120,0.2,"""ifrs4""","""comision_cesion""","""reaseguro_proporcional""","""recibo""",-1,0,2024-01-15,2024-01-01,2024-01-15,2025-01-15,48.0
"""anual_misma_vigencia""","""comision_rea_prop""",1111,"""mantenido""",3693358,"""ext""",0.3,1944219246,2023-07-01,"""02""","""083""",124324091,"""INVALIDEZ""",124324511,"""banca""","""plan_3""","""USD""",2024-01-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,120,0.2,"""ifrs17_corporativo""","""comision_cesion""","""reaseguro_proporcional""","""cobertura""",-1,0,2024-01-15,2024-01-01,2024-01-15,2025-01-15,36.0
"""anual_misma_vigencia""","""comision_rea_prop""",1111,"""mantenido""",3693358,"""ext""",0.3,1944219246,2023-07-01,"""02""","""083""",124324091,"""INVALIDEZ""",124324511,"""banca""","""plan_3""","""USD""",2024-01-01,2024-01-15,2025-01-15,2024-01-15,2025-01-15,120,0.2,"""ifrs17_local""","""comision_cesion""","""reaseguro_proporcional""","""cobertura""",-1,0,2024-01-15,2024-01-01,2024-01-15,2025-01-15,36.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""mensual_mes_fin""","""comision_rea_prop""",1245,"""mantenido""",3693358,"""ext""",0.5,5260869017,2024-01-01,"""01""","""040""",345675387,"""HURTO""",345675807,"""sucursal""","""plan_2""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,6,0.1,"""ifrs17_local""","""comision_cesion""","""reaseguro_proporcional""","""cobertura""",-1,0,2024-01-15,2024-02-20,2024-02-20,2024-02-15,3.0
"""mensual_mes_fin""","""comision_rea_prop""",1245,"""mantenido""",3693358,"""ext""",0.5,5260869017,2024-01-01,"""01""","""040""",345675387,"""HURTO""",345675807,"""sucursal""","""plan_2""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,6,0.1,"""ifrs4""","""comision_cesion""","""reaseguro_proporcional""","""recibo""",-1,0,2024-01-15,2024-02-20,2024-02-20,2024-02-15,3.0
"""mensual_mes_fin""","""comision_rea_prop""",1245,"""mantenido""",2374188,"""int""",0.5,1292794322,2024-01-01,"""01""","""040""",345675387,"""HURTO""",345675807,"""sucursal""","""plan_2""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,6,0.1,"""ifrs17_corporativo""","""comision_cesion""","""reaseguro_pr

## 1.3. PCR del Reaseguro No Proporcional

### Componente Costo Contrato del reaseguro No Proporcional

In [9]:
costo_contrato_rea

tipo_insumo,contrato_reaseguro,tipo_negocio,fe_ini_vig_contrato_reaseguro,fe_fin_vig_contrato_reaseguro,nit_reasegurador,tipo_reasegurador,porc_participacion_reasegurador,compania,ramo_sura,producto,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,fe_reinstalamento,recibo_costo_contrato,fecha_contabilizacion_recibo,moneda,valor_costo_contrato
str,i64,str,date,date,i64,str,f64,str,str,str,date,date,date,date,date,i64,date,str,i64
"""costo_contrato_rea_noprop""",292161,"""mantenido""",2025-06-30,2024-07-01,507351248,"""int""",0.2,"""02""","""090""","""plan1""",2024-07-01,2025-06-30,2024-07-01,2025-06-30,null,47628239,2024-07-01,"""COP""",7000000000
"""costo_contrato_rea_noprop""",292161,"""mantenido""",2025-06-30,2024-07-01,368484453,"""ext""",0.8,"""02""","""090""","""plan1""",2024-07-01,2025-06-30,2024-07-01,2025-06-30,null,47628239,2024-07-01,"""COP""",7000000000
"""costo_contrato_rea_noprop""",373318,"""mantenido""",2025-06-30,2024-07-01,446526767,"""int""",1.0,"""01""","""009""","""plan1""",2024-07-01,2025-06-30,2024-07-01,2025-06-30,null,71800418,2024-07-01,"""COP""",5350000000
"""costo_contrato_rea_noprop""",867346,"""mantenido""",2025-12-31,2025-01-01,325775298,"""int""",1.0,"""01""","""037""","""plan1""",2025-01-01,2025-12-31,2025-01-01,2025-12-31,null,45927212,2025-01-01,"""COP""",134000000
"""costo_contrato_rea_noprop""",867346,"""mantenido""",2025-12-31,2025-01-01,360681631,"""int""",1.0,"""01""","""037""","""plan1""",2025-01-01,2025-12-31,2025-01-01,2025-12-31,2025-06-03,45927219,2025-06-15,"""COP""",90000000


In [10]:
# prepara el insumo para el devengo
input_costcon_rea = prep_data.prep_input_costo_con(
        costo_contrato_rea, 
        seguimiento_rea, 
        param_contab, 
        excepciones, 
        FECHA_VALORACION
    )
input_costcon_rea

tipo_insumo,contrato_reaseguro,tipo_negocio,fe_ini_vig_contrato_reaseguro,fe_fin_vig_contrato_reaseguro,nit_reasegurador,tipo_reasegurador,porc_participacion_reasegurador,compania,ramo_sura,producto,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,fe_reinstalamento,recibo_costo_contrato,fecha_contabilizacion_recibo,moneda,valor_costo_contrato,tipo_contabilidad,componente,tipo_contrato,nivel_detalle,signo_constitucion,candidato_devengo_50_50,limite_agregado_valor_instalado,valor_siniestros_incurridos_mes,limite_agregado_casos_instalado,casos_incurridos_mes,fecha_inicio_vigencia,fecha_constitucion,fecha_inicio_devengo,fecha_fin_devengo,valor_base_devengo
str,i64,str,date,date,i64,str,f64,str,str,str,date,date,date,date,date,i64,date,str,i64,str,str,str,str,i64,i32,i64,i64,i64,i64,date,date,date,date,f64
"""costo_contrato_rea_noprop""",292161,"""mantenido""",2025-06-30,2024-07-01,507351248,"""int""",0.2,"""02""","""090""","""plan1""",2024-07-01,2025-06-30,2024-07-01,2025-06-30,null,47628239,2024-07-01,"""COP""",7000000000,"""ifrs17_corporativo""","""costo_contrato""","""reaseguro_no_proporcional""","""recibo""",1,0,10000000000,1658491901,100,15,2024-07-01,2024-07-01,2024-07-01,2025-06-30,1.4000e9
"""costo_contrato_rea_noprop""",292161,"""mantenido""",2025-06-30,2024-07-01,368484453,"""ext""",0.8,"""02""","""090""","""plan1""",2024-07-01,2025-06-30,2024-07-01,2025-06-30,null,47628239,2024-07-01,"""COP""",7000000000,"""ifrs17_corporativo""","""costo_contrato""","""reaseguro_no_proporcional""","""recibo""",1,0,10000000000,1658491901,100,15,2024-07-01,2024-07-01,2024-07-01,2025-06-30,5.6000e9
"""costo_contrato_rea_noprop""",373318,"""mantenido""",2025-06-30,2024-07-01,446526767,"""int""",1.0,"""01""","""009""","""plan1""",2024-07-01,2025-06-30,2024-07-01,2025-06-30,null,71800418,2024-07-01,"""COP""",5350000000,"""ifrs17_corporativo""","""costo_contrato""","""reaseguro_no_proporcional""","""recibo""",1,0,50000000000,3221449571,50,5,2024-07-01,2024-07-01,2024-07-01,2025-06-30,5.3500e9
"""costo_contrato_rea_noprop""",292161,"""mantenido""",2025-06-30,2024-07-01,507351248,"""int""",0.2,"""02""","""090""","""plan1""",2024-07-01,2025-06-30,2024-07-01,2025-06-30,null,47628239,2024-07-01,"""COP""",7000000000,"""ifrs17_local""","""costo_contrato""","""reaseguro_no_proporcional""","""cobertura""",1,0,10000000000,1658491901,100,15,2024-07-01,2024-07-01,2024-07-01,2025-06-30,1.4000e9
"""costo_contrato_rea_noprop""",292161,"""mantenido""",2025-06-30,2024-07-01,368484453,"""ext""",0.8,"""02""","""090""","""plan1""",2024-07-01,2025-06-30,2024-07-01,2025-06-30,null,47628239,2024-07-01,"""COP""",7000000000,"""ifrs17_local""","""costo_contrato""","""reaseguro_no_proporcional""","""cobertura""",1,0,10000000000,1658491901,100,15,2024-07-01,2024-07-01,2024-07-01,2025-06-30,5.6000e9
"""costo_contrato_rea_noprop""",373318,"""mantenido""",2025-06-30,2024-07-01,446526767,"""int""",1.0,"""01""","""009""","""plan1""",2024-07-01,2025-06-30,2024-07-01,2025-06-30,null,71800418,2024-07-01,"""COP""",5350000000,"""ifrs17_local""","""costo_contrato""","""reaseguro_no_proporcional""","""cobertura""",1,0,50000000000,3221449571,50,5,2024-07-01,2024-07-01,2024-07-01,2025-06-30,5.3500e9


# 2. Cálculo del Devengamiento

- El devengamiento se realiza mediante una única función 'devengar' porque los cálculos esenciales son los mismos.
- Se puede devengar un insumo solamente como en 'output_solo_primadir'
- También se pueden devengar varios o todos los insumos preparados en un solo paso como en 'output_todos'
- Se obtiene una tabla con el detalle de los inputs cuando aplique (nulo cuando no aplique) y los outputs de flujos de reserva
- Los signos del output han sido ajustados con el parametro signo_constitucion para reflejar la dirección en que debe aplicar el movimiento de constitución o liberación (- es pasivo y + representa un activo desde el punto de vista de la aseguradora). Este signo es independiente del signo del valor del recibo, que debe venir de acuerdo al tipo de movimiento (- para cancelaciones/mod valorables negativas y + para emisiones). Si los signos de comisiones, costo contrato o primas cedidas llegan con un negativo que representa un egreso, deberían adaptarse los signos.

### Devengamiento de un solo insumo preparado: prima del seguro directo
Existen varias reglas de devengamiento según el tipo de insumo:
1. Devengamiento diario uniforme: es la regla general, cada día de riesgo corrido tiene el mismo valor y se libera por el número de días transcurridos en cada periodo
2. Devengamiento 50/50: es una regla especial ajustada a la normativa local bajo IFRS4 que requiere que se mantenga una reserva del 50% del valor de la prima durante las vigencias mensuales (menores o iguales a 32 días de vigencia).
3. Devengamiento por consumo del límite agregado del contrato no proporcional

In [11]:
# output del devengamiento para un solo tipo de insumo preparado
output_solo_primadir = devg.devengar(input_prod_dir, FECHA_VALORACION)
output_solo_primadir

aux_casuistica,tipo_insumo,tipo_negocio,poliza,fecha_expedicion_poliza,recibo,amparo,poliza_certificado,compania,ramo_sura,canal,producto,tipo_op,moneda,fecha_contabilizacion_recibo,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,valor_prima_emitida,nivel_detalle,signo_constitucion,candidato_devengo_50_50,fecha_inicio_vigencia,fecha_valoracion_anterior,saldo_anterior,componente,tipo_contabilidad,regla_devengo,tipo_contrato,cohorte,anio_liberacion,transicion,fecha_inicio_periodo,fecha_valoracion,fecha_constitucion,fecha_inicio_devengo,fecha_fin_devengo,valor_base_devengo,dias_devengados,dias_no_devengados,control_suma_dias,dias_constitucion,dias_liberacion,valor_devengo_diario,mes_constitucion,mes_ini_liberacion,mes_fin_liberacion,estado_devengo,valor_constitucion,valor_liberacion,valor_liberacion_acum,saldo
str,str,str,i64,date,i64,str,i64,str,str,str,str,str,str,date,date,date,date,date,f64,str,i64,i32,date,date,f64,str,str,str,str,i32,str,i32,date,date,date,date,date,f64,i64,i64,bool,i64,i64,f64,i32,i32,i32,str,f64,f64,f64,f64
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""cobertura""",-1,0,2024-01-30,2024-11-30,0.0,"""prima""","""ifrs17_local""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-31,2024-01-30,2024-01-30,2024-02-15,50.0,0,0,true,0,0,0.0,null,null,null,"""finalizado""",-0.0,0.0,0.0,-0.0
"""mensual_mes_fin""","""produccion_directo""","""directo""",345675387,2024-02-20,2938431543,"""HURTO""",345675807,"""01""","""040""","""sucursal""","""plan_2""","""emision""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100.0,"""cobertura""",-1,0,2024-01-15,2024-11-30,-100.0,"""prima""","""ifrs17_local""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-31,2024-02-20,2024-02-20,2024-02-15,100.0,352,0,false,32,32,3.125,null,null,null,"""entra_devengado""",-0.0,100.0,1100.0,-0.0
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""recibo""",-1,0,2024-01-30,2024-11-30,0.0,"""prima""","""ifrs4""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-31,2024-01-30,2024-01-30,2024-02-15,50.0,0,0,true,0,0,0.0,null,null,null,"""finalizado""",-0.0,0.0,0.0,-0.0
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""cobertura""",-1,0,2024-01-30,2024-11-30,0.0,"""prima""","""ifrs17_corporativo""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-31,2024-01-30,2024-01-30,2024-02-15,50.0,0,0,true,0,0,0.0,null,null,null,"""finalizado""",-0.0,0.0,0.0,-0.0
"""mensual_mes_fin""","""produccion_directo""","""directo""",345675387,2024-02-20,2938431543,"""HURTO""",345675807,"""01""","""040""","""sucursal""","""plan_2""","""emision""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100.0,"""cobertura""",-1,0,2024-01-15,2024-11-30,-100.0,"""prima""","""ifrs17_corporativo""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-31,2024-02-20,2024-02-20,2024-02-15,100.0,352,0,false,32,32,3.125,null,null,null,"""entra_devengado""",-0.0,100.0,1100.0,-0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""mensual_en_vigencia""","""produccion_directo""","""directo""",552332523,2024-01-15,2938434015,"""MEDICINA""",552332943,"""02""",

### Devengamiento de varios insumos preparados: todos a la vez

- El esquema o tipos de datos deben estar alineados entre los insumos
- Se concatenan verticalmente (UNION ALL)
- Se le entregan al motor de devengo y este devuelve el output para todas las entradas consolidado
- Primero aparecen todos los datos de entrada según el tipo de insumo y luego los datos de salida
- Algunos datos de salida solo aplican para el devengamiento diario y otros solo aplican para el devengamiento regla 50/50, por lo cual aparecen campos nulos, pero los campos de movimientos de constitución y liberación y saldo se calculan para ambas reglas y no pueden ser vacíos.
- El output se puede ver en excel en: LINK OUTPUT DETERIORO

In [12]:
# En lugar de pasar un solo input preparado, se le pasan varios o todos a la vez
insumos_devengo = [
        input_prod_dir, 
        input_gasto_dir, 
        input_dcto_dir, 
        input_cesion_rea, 
        input_dcto_rea, 
        input_gasto_rea,
        input_comi_rea,
        input_costcon_rea
    ]
input_consolidado = pl.concat(aux_tools.alinear_esquemas(insumos_devengo), how='diagonal')
output_todos = devg.devengar(input_consolidado, FECHA_VALORACION)
output_todos

aux_casuistica,tipo_insumo,tipo_negocio,poliza,fecha_expedicion_poliza,recibo,amparo,poliza_certificado,compania,ramo_sura,canal,producto,tipo_op,moneda,fecha_contabilizacion_recibo,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,valor_prima_emitida,nivel_detalle,signo_constitucion,candidato_devengo_50_50,fecha_inicio_vigencia,tipo_gasto,porc_gasto,prioridad_match,rn,podto_comercial,podto_tecnico,contrato_reaseguro,nit_reasegurador,tipo_reasegurador,porc_participacion_reasegurador,fe_ini_vig_contrato_reaseguro,fe_fin_vig_contrato_reaseguro,valor_prima_cedida,…,recibo_costo_contrato,valor_costo_contrato,limite_agregado_valor_instalado,valor_siniestros_incurridos_mes,limite_agregado_casos_instalado,casos_incurridos_mes,fecha_valoracion_anterior,valor_liberacion_limite,saldo_anterior,componente,tipo_contabilidad,regla_devengo,tipo_contrato,cohorte,anio_liberacion,transicion,fecha_inicio_periodo,fecha_valoracion,fecha_constitucion,fecha_inicio_devengo,fecha_fin_devengo,valor_base_devengo,dias_devengados,dias_no_devengados,control_suma_dias,dias_constitucion,dias_liberacion,valor_devengo_diario,mes_constitucion,mes_ini_liberacion,mes_fin_liberacion,porc_consumo_limite,estado_devengo,valor_constitucion,valor_liberacion,valor_liberacion_acum,saldo
str,str,str,i64,date,i64,str,i64,str,str,str,str,str,str,date,date,date,date,date,f64,str,i64,i32,date,str,f64,i32,i64,f64,f64,i64,i64,str,f64,date,date,i64,…,i64,i64,i64,i64,i64,i64,date,f64,f64,str,str,str,str,i32,str,i32,date,date,date,date,date,f64,i64,i64,bool,i64,i64,f64,i32,i32,i32,f64,str,f64,f64,f64,f64
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""cobertura""",-1,0,2024-01-30,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,2024-11-30,null,0.0,"""prima""","""ifrs17_local""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-31,2024-01-30,2024-01-30,2024-02-15,50.0,0,0,true,0,0,0.0,null,null,null,null,"""finalizado""",-0.0,0.0,0.0,-0.0
"""mensual_mes_fin""","""produccion_directo""","""directo""",345675387,2024-02-20,2938431543,"""HURTO""",345675807,"""01""","""040""","""sucursal""","""plan_2""","""emision""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100.0,"""cobertura""",-1,0,2024-01-15,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,2024-11-30,null,-100.0,"""prima""","""ifrs17_local""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-31,2024-02-20,2024-02-20,2024-02-15,100.0,352,0,false,32,32,3.125,null,null,null,null,"""entra_devengado""",-0.0,100.0,1100.0,-0.0
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""recibo""",-1,0,2024-01-30,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,2024-11-30,null,0.0,"""prima""","""ifrs4""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-31,2024-01-30,2024-01-30,2024-02-15,50.0,0,0,true,0,0,0.0,null,null,null,null,"""finalizado""",-0.0,0.0,0.0,-0.0
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""cobertura""",-1,0,2024-01-30,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,2024-11-30,null,0.0,"""prima""","""ifrs17_corpo

### Devengamiento del Costo de Contrato de Reaseguro No Proporcional

El devengamiento de este componente sigue una regla adicional especial para ajustarse a los requerimientos de IFRS 17.

La regla de devengamiento se debe elegir mensualmente dependiendo de la comparación entre el valor consumido del límite agregado del contrato de reaseguro y el valor consumido según el devengamiento diario:

$$
\text{liberación}_t = 
\begin{cases}
\max\left(\text{liberación}_t^{\text{diario}}, \text{liberación}_t^{\text{límite}}\right) & \text{si } 0 <= t < T \\
\text{saldo}_{t-1} & \text{si } t = T
\end{cases}
$$

La liberación por devengamiento diario se debe recalcular en cada periodo durante la vigencia porque, si en un periodo anterior se devengó por consumo del límite agregado, el valor de devengo diario (prima diaria) original ya no será válido y debe ajustarse en función del saldo restante por devengar y de la duración restante de la vigencia, así:

$$
\text{liberación}_t^{\text{diario}} = \frac{\text{saldo}_{t-1}}{\text{días}_{t-1}^{\text{restantes}}} \times \text{días}_t^{\text{mes}}
$$

Por su parte, el devengamiento según el consumo del límite agregado del contrato se calcula como la proporción del límite agregado que ha sido consumida por siniestros incurridos en el periodo de valoración $t$, aplicada al saldo remanente del contrato al inicio del periodo $t$. Esta aproximación busca reflejar el patrón real de servicio recibido, alineado con el grado de exposición al riesgo efectivamente utilizado por el asegurado en ese mes.

$$
\text{liberación}_t^{\text{límite}} = \left( \frac{\text{siniestros incurridos}_t}{\text{límite agregado}_t} \right) \times \text{saldo}_{t-1}
$$

En el primer periodo de devengamiento y durante la vigencia se comparan las dos alternativas de liberación y se elige el método que resulte en el mayor valor liberado, los saldos de reserva deben ajustarse a la liberación realizada.
En el último periodo, siempre se debe garantizar que se libere todo el saldo que queda en la reserva.

Donde:
- $\text{saldo}_{t-1}$: es el saldo por devengar al final del mes anterior o inicio del mes t.
- $\text{días}_{t-1}^{\text{restantes}}$: número de días de vigencia restantes al inicio del mes $t$.
- $\text{días}_t^{\text{mes}}$: número de días del mes $t$.
- $T$: último periodo de vigencia.
- $\text{siniestros acumulados}_t$: valor acumulado de los siniestros incurridos cubiertos por el contrato de reaseguro hasta el mes $t$.
- $\text{límite agregado}_t$: valor máximo de cobertura del contrato de reaseguro, puede cambiar si se dan reinstalamentos.


In [13]:
# La función devengar incorpora la lógica descrita
output_costo_contrato = devg.devengar(input_costcon_rea, FECHA_VALORACION)
output_costo_contrato

tipo_insumo,contrato_reaseguro,tipo_negocio,fe_ini_vig_contrato_reaseguro,fe_fin_vig_contrato_reaseguro,nit_reasegurador,tipo_reasegurador,porc_participacion_reasegurador,compania,ramo_sura,producto,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,fe_reinstalamento,recibo_costo_contrato,fecha_contabilizacion_recibo,moneda,valor_costo_contrato,nivel_detalle,signo_constitucion,candidato_devengo_50_50,limite_agregado_valor_instalado,valor_siniestros_incurridos_mes,limite_agregado_casos_instalado,casos_incurridos_mes,fecha_inicio_vigencia,fecha_valoracion_anterior,valor_liberacion_limite,saldo_anterior,componente,tipo_contabilidad,regla_devengo,tipo_contrato,cohorte,anio_liberacion,transicion,fecha_inicio_periodo,fecha_valoracion,fecha_constitucion,fecha_inicio_devengo,fecha_fin_devengo,valor_base_devengo,dias_devengados,dias_no_devengados,control_suma_dias,dias_constitucion,dias_liberacion,valor_devengo_diario,porc_consumo_limite,estado_devengo,valor_constitucion,valor_liberacion,valor_liberacion_acum,saldo
str,i64,str,date,date,i64,str,f64,str,str,str,date,date,date,date,date,i64,date,str,i64,str,i64,i32,i64,i64,i64,i64,date,date,f64,f64,str,str,str,str,i32,str,i32,date,date,date,date,date,f64,i64,i64,bool,i64,i64,f64,f64,str,f64,f64,f64,f64
"""costo_contrato_rea_noprop""",292161,"""mantenido""",2025-06-30,2024-07-01,507351248,"""int""",0.2,"""02""","""090""","""plan1""",2024-07-01,2025-06-30,2024-07-01,2025-06-30,null,47628239,2024-07-01,"""COP""",7000000000,"""recibo""",1,0,10000000000,1658491901,100,15,2024-07-01,2024-11-30,2.3219e8,9.3562e8,"""costo_contrato""","""ifrs17_corporativo""","""consumo_limite""","""reaseguro_no_proporcional""",2025,"""anio_actual""",0,2024-12-01,2024-12-31,2024-07-01,2024-07-01,2025-06-30,1.4000e9,184,181,true,365,31,3.8356e6,0.165849,"""en_curso""",0.0,-2.3219e8,-2.3219e8,1.1678e9
"""costo_contrato_rea_noprop""",292161,"""mantenido""",2025-06-30,2024-07-01,368484453,"""ext""",0.8,"""02""","""090""","""plan1""",2024-07-01,2025-06-30,2024-07-01,2025-06-30,null,47628239,2024-07-01,"""COP""",7000000000,"""recibo""",1,0,10000000000,1658491901,100,15,2024-07-01,2024-11-30,9.2876e8,3.7425e9,"""costo_contrato""","""ifrs17_corporativo""","""consumo_limite""","""reaseguro_no_proporcional""",2025,"""anio_actual""",0,2024-12-01,2024-12-31,2024-07-01,2024-07-01,2025-06-30,5.6000e9,184,181,true,365,31,1.5342e7,0.165849,"""en_curso""",0.0,-9.2876e8,-9.2876e8,4.6712e9
"""costo_contrato_rea_noprop""",373318,"""mantenido""",2025-06-30,2024-07-01,446526767,"""int""",1.0,"""01""","""009""","""plan1""",2024-07-01,2025-06-30,2024-07-01,2025-06-30,null,71800418,2024-07-01,"""COP""",5350000000,"""recibo""",1,0,50000000000,3221449571,50,5,2024-07-01,2024-11-30,3.4470e8,4.4412e9,"""costo_contrato""","""ifrs17_corporativo""","""diario""","""reaseguro_no_proporcional""",2025,"""anio_actual""",0,2024-12-01,2024-12-31,2024-07-01,2024-07-01,2025-06-30,5.3500e9,184,181,true,365,31,1.4658e7,0.064429,"""en_curso""",0.0,-4.5438e8,-4.5438e8,4.8956e9
"""costo_contrato_rea_noprop""",292161,"""mantenido""",2025-06-30,2024-07-01,507351248,"""int""",0.2,"""02""","""090""","""plan1""",2024-07-01,2025-06-30,2024-07-01,2025-06-30,null,47628239,2024-07-01,"""COP""",7000000000,"""cobertura""",1,0,10000000000,1658491901,100,15,2024-07-01,2024-11-30,2.3219e8,9.3562e8,"""costo_contrato""","""ifrs17_local""","""consumo_limite""","""reaseguro_no_proporcional""",2025,"""anio_actual""",0,2024-12-01,2024-12-31,2024-07-01,2024-07-01,2025-06-30,1.4000e9,184,181,true,365,31,3.8356e6,0.165849,"""en_curso""",0.0,-2.3219e8,-2.3219e8,1.1678e9
"""costo_contrato_rea_noprop""",292161,"""mantenido""",2025-06-30,2024-07-01,368484453,"""ext""",0.8,"""02""","""090""","""plan1""",2024-07-01,2025-06-30,2024-07-01,2025-06-30,null,47628239,2024-07-01,"""COP""",7000000000,"""cobertura""",1,0,10000000000,1658491901,100,15,2024-07-01,2024-11-30,9.2876e8,3.7425e9,"""costo_contrato""","""ifrs17_local""",""

# 3. Fluctuación

La fluctuación aplica solo para los componentes expresados en moneda extranjera, por eso primero identifica cuáles líneas del output requieren este cálculo y, de no requerirlo, la fluctuación será cero:

$$
\text{Fluc} =
\begin{cases}
0 & \text{si } \text{moneda} = \text{COP} \\
\text{Fluc}_{\text{liberación}} + \text{Fluc}_{\text{constitución}} & \text{en otro caso}
\end{cases}
$$


## Fluctuación IFRS4, IFRS17 Local y Corporativo:

- Se debe fluctuar el valor completo del saldo de reserva que la aseguradora mantuvo durante el mes de medición. Por ello, la base de fluctuación será el saldo de apertura del mes contable actual, es decir $ \text{saldo}_{t-1} $ y en los casos en que el mes se haya aperturado sin saldo, pero se haya realizado una constitución, la base será la constitución realizada en el periodo.

$$
\text{Fluc}_t =
\begin{cases}
\text{constitución}_t \times (\text{TC}_t - \text{TC}_{{\text{bautizo}}}) & \text{si } t = 0 \\
\text{saldo}_{t-1} \times (\text{TC}_t - \text{TC}_{t-1}) & \text{si } 0 < t \leq T \\
\end{cases}
$$

- La constitución y la liberación se fluctúan por separado:
  - La base para la **fluctuación de la liberación** es el negativo de la liberación en el mes $ t $.
  - La base para la **fluctuación de la constitución** será el valor del saldo final del mes actual $ \text{saldo}_t $, el cual refleja el valor que permanece constituido al cierre del mes.
  - En el primer mes, la tasa de bautizo representa la tasa exacta de reconocimiento del pasivo. En meses posteriores, se usa la tasa del cierre anterior para evitar doble contabilización de la variación cambiaria.

En resumen:

$$
\text{Fluc}_{\text{liberación}} =
\begin{cases}
-\text{liberación}_t \times (\text{TC}_t - \text{TC}_{{\text{bautizo}}}) & \text{si } t = 0 \\
-\text{liberación}_t \times (\text{TC}_t - \text{TC}_{t-1}) & \text{si } 0 < t \leq T \\
\end{cases}
$$

$$
\text{Fluc}_{\text{constitución}} =
\begin{cases}
\text{saldo}_t \times (\text{TC}_t - \text{TC}_{{\text{bautizo}}}) & \text{si } t = 0 \\
\text{saldo}_{t} \times (\text{TC}_t - \text{TC}_{t-1}) & \text{si } 0 < t \leq T \\
\end{cases}
$$

Notación:

- $ t $: cierre del mes contable actual.  
- $ t-1 $: cierre del mes contable anterior.  
- $ T $: cierre del mes en que se liquida el saldo o se libera completamente.  
- $ \text{TC}_t $: tasa de cambio al cierre del mes contable $ t $ o fecha de valoración.  
- $ \text{TC}_{\text{bautizo}} $: tasa de cambio del día exacto de contabilización del recibo.  

---

### Implicaciones de signo en la fluctuación:

La fluctuación, tanto de constitución, como de liberación **debe reflejar siempre el signo del efecto económico de la diferencia en tasas de cambio**. Este efecto depende de la naturaleza de la reserva (activo o pasivo) y de la dirección del cambio en tasa:

Cuando la PCR es un pasivo (como en el seguro directo):
  - Un aumento en la tasa de cambio genera fluctuación **negativa** porque incrementa el valor de la obligación.
  - Una disminución de la tasa de cambio reduce el valor de la obligación de la aseguradora, generando un efecto contable **positivo.**

Cuando la naturaleza de la reserva es la de un activo, como en la PCR del reaseguro cedido, ocurre lo opuesto y el signo de la fluctuación es **positivo** cuando la tasa incrementa y **negativo** cuando la tasa disminuye.

---

### Fluctuación y Saldo de la PCR en COP:

Para reconstruir el saldo de la reserva en moneda local a partir de la fluctuación:
1. Se convierten los movimientos de constitución y liberación a COP con la tasa de cambio correspondiente.
2. Se recalcula el saldo como:
$$ saldo_t^{COP} = saldo_{t-1}^{COP} + liberación_t^{COP} + fluctuación_{t}^{COP} $$
Donde:
La liberación $ liberación_{t}^{COP} $ es positiva si la naturaleza de la reserva es pasivo (seguro directo) y negativa si es activo (cedido) y $fluctuación_{t}^{COP}$ siempre tiene el signo según la naturaleza de la reserva y la dirección del cambio en tasas.


In [14]:
# devuelve la base ya devengada, con las columnas adicionales de fluctuación
output_fluct = fluc.calc_fluctuacion(output_todos, tasa_cambio)
output_fluct

aux_casuistica,tipo_insumo,tipo_negocio,poliza,fecha_expedicion_poliza,recibo,amparo,poliza_certificado,compania,ramo_sura,canal,producto,tipo_op,moneda,fecha_contabilizacion_recibo,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,valor_prima_emitida,nivel_detalle,signo_constitucion,candidato_devengo_50_50,fecha_inicio_vigencia,tipo_gasto,porc_gasto,prioridad_match,rn,podto_comercial,podto_tecnico,contrato_reaseguro,nit_reasegurador,tipo_reasegurador,porc_participacion_reasegurador,fe_ini_vig_contrato_reaseguro,fe_fin_vig_contrato_reaseguro,valor_prima_cedida,…,casos_incurridos_mes,fecha_valoracion_anterior,valor_liberacion_limite,saldo_anterior,componente,tipo_contabilidad,regla_devengo,tipo_contrato,cohorte,anio_liberacion,transicion,fecha_inicio_periodo,fecha_valoracion,fecha_constitucion,fecha_inicio_devengo,fecha_fin_devengo,valor_base_devengo,dias_devengados,dias_no_devengados,control_suma_dias,dias_constitucion,dias_liberacion,valor_devengo_diario,mes_constitucion,mes_ini_liberacion,mes_fin_liberacion,porc_consumo_limite,estado_devengo,valor_constitucion,valor_liberacion,valor_liberacion_acum,saldo,tasa_cambio_fecha_valoracion,tasa_cambio_fecha_valoracion_anterior,tasa_cambio_fecha_constitucion,fluctuacion_constitucion,fluctuacion_liberacion
str,str,str,i64,date,i64,str,i64,str,str,str,str,str,str,date,date,date,date,date,f64,str,i64,i32,date,str,f64,i32,i64,f64,f64,i64,i64,str,f64,date,date,i64,…,i64,date,f64,f64,str,str,str,str,i32,str,i32,date,date,date,date,date,f64,i64,i64,bool,i64,i64,f64,i32,i32,i32,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""cobertura""",-1,0,2024-01-30,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,2024-11-30,null,0.0,"""prima""","""ifrs17_local""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-31,2024-01-30,2024-01-30,2024-02-15,50.0,0,0,true,0,0,0.0,null,null,null,null,"""finalizado""",-0.0,0.0,0.0,-0.0,1.0,1.0,1.0,0.0,0.0
"""mensual_mes_fin""","""produccion_directo""","""directo""",345675387,2024-02-20,2938431543,"""HURTO""",345675807,"""01""","""040""","""sucursal""","""plan_2""","""emision""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100.0,"""cobertura""",-1,0,2024-01-15,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,2024-11-30,null,-100.0,"""prima""","""ifrs17_local""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-31,2024-02-20,2024-02-20,2024-02-15,100.0,352,0,false,32,32,3.125,null,null,null,null,"""entra_devengado""",-0.0,100.0,1100.0,-0.0,1.0,1.0,1.0,0.0,0.0
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""recibo""",-1,0,2024-01-30,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,2024-11-30,null,0.0,"""prima""","""ifrs4""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-31,2024-01-30,2024-01-30,2024-02-15,50.0,0,0,true,0,0,0.0,null,null,null,null,"""finalizado""",-0.0,0.0,0.0,-0.0,1.0,1.0,1.0,0.0,0.0
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""cobertura""",-1,0,2024-01-30,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,2024-11-30,null,0.0,"""prima""","""ifrs17_corporativo""","""diario""","""directo

# 4. Deterioro [Solo Reaseguro]

El deterioro es un proceso que aplica únicamente a las reservas de Reaseguro. El objetivo es determinar el valor con una alta probabilidad de incobrabilidad por riesgo de crédito asociado al reasegurador. Es decir, calcular una pérdida por la probabilidad de que el reasegurador no pueda responder por sus obligaciones futuras valoradas en la reserva PCR.

- Solo aplica a reaseguro cuando la posición de la reserva PCR es activa, es decir reaseguro mantenido (cedido) y reaseguro retrocedido.
- Debe ser trazable la fecha de valoración y probabilidad de default (incumplimiento) por reasegurador $t$ con las cuales se calcula el deterioro

$$
\text{deterioro}_t^{r} = \text{saldo}_t^{r} \times \text{PD}_t^r
$$

Donde: 
- $r$ reasegurador
- $t$ es la fecha de valoración
- $PD_t^r$ probabilidad de incumplimiento del reasegurador r en el momento t(Fuente Riesgo Crédito)

La contabilización del deterioro se hace en movimientos. 

1. **Constitución Deterioro:** se constituye deterioro del saldo final de la PCR por un incremento en probabilidad de incumplimiento
$$
\text{constitucion deterioro}_t^{r} = saldo_t^r \times \max(\text{PD}_{t}^{r} - \text{PD}_{t-1}^{r}, 0)
$$
2. **Liberación Deterioro:** se pueden producir liberaciones de deterioro en 2 casos que pueden ocurrir a la vez:
     - El saldo de la PCR se reduce respecto al periodo anterior
     - La probabilidad de incumplimiento del reasegurador disminuye respecto al periodo anterior
    
$$
\text{liberacion deterioro}_t^{r} = \min(\text{saldo}_{t}^{r} - \text{saldo}_{t-1}^{r}, 0) \times \text{PD}_{t-1}^{r} + saldo_{t-1}^r \times \min(\text{PD}_{t}^{r} - \text{PD}_{t-1}^{r}, 0)
$$

El valor de cada movimiento debe calcularse en moneda original del documento

In [15]:
import src.deterioro as det

# Calcula el output del deterioro con movimientos del mes y el historico actualizado
output_deterioro= det.calc_deterioro(
    output_devengo_fluc=output_fluct, 
    riesgo_credito=input_riesgo_credito,
    fe_valoracion=FECHA_VALORACION
)
# El output con movimientos de deterioro
output_deterioro

(169, 85) (70, 85) (70, 88)
(169, 85) (70, 85) (99, 90) (70, 90)


aux_casuistica,tipo_insumo,tipo_negocio,poliza,fecha_expedicion_poliza,recibo,amparo,poliza_certificado,compania,ramo_sura,canal,producto,tipo_op,moneda,fecha_contabilizacion_recibo,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,valor_prima_emitida,nivel_detalle,signo_constitucion,candidato_devengo_50_50,fecha_inicio_vigencia,tipo_gasto,porc_gasto,prioridad_match,rn,podto_comercial,podto_tecnico,contrato_reaseguro,nit_reasegurador,tipo_reasegurador,porc_participacion_reasegurador,fe_ini_vig_contrato_reaseguro,fe_fin_vig_contrato_reaseguro,valor_prima_cedida,…,tipo_contabilidad,regla_devengo,tipo_contrato,cohorte,anio_liberacion,transicion,fecha_inicio_periodo,fecha_valoracion,fecha_constitucion,fecha_inicio_devengo,fecha_fin_devengo,valor_base_devengo,dias_devengados,dias_no_devengados,control_suma_dias,dias_constitucion,dias_liberacion,valor_devengo_diario,mes_constitucion,mes_ini_liberacion,mes_fin_liberacion,porc_consumo_limite,estado_devengo,valor_constitucion,valor_liberacion,valor_liberacion_acum,saldo,tasa_cambio_fecha_valoracion,tasa_cambio_fecha_valoracion_anterior,tasa_cambio_fecha_constitucion,fluctuacion_constitucion,fluctuacion_liberacion,prob_incumplimiento_actual,prob_incumplimiento_anterior,cambio_prob_incumplimiento,constitucion_deterioro,liberacion_deterioro
str,str,str,i64,date,i64,str,i64,str,str,str,str,str,str,date,date,date,date,date,f64,str,i64,i32,date,str,f64,i32,i64,f64,f64,i64,i64,str,f64,date,date,i64,…,str,str,str,i32,str,i32,date,date,date,date,date,f64,i64,i64,bool,i64,i64,f64,i32,i32,i32,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""cobertura""",-1,0,2024-01-30,null,null,null,null,null,null,null,null,null,null,null,null,null,…,"""ifrs17_local""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-31,2024-01-30,2024-01-30,2024-02-15,50.0,0,0,true,0,0,0.0,null,null,null,null,"""finalizado""",-0.0,0.0,0.0,-0.0,1.0,1.0,1.0,0.0,0.0,null,null,null,null,null
"""mensual_mes_fin""","""produccion_directo""","""directo""",345675387,2024-02-20,2938431543,"""HURTO""",345675807,"""01""","""040""","""sucursal""","""plan_2""","""emision""","""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,100.0,"""cobertura""",-1,0,2024-01-15,null,null,null,null,null,null,null,null,null,null,null,null,null,…,"""ifrs17_local""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-31,2024-02-20,2024-02-20,2024-02-15,100.0,352,0,false,32,32,3.125,null,null,null,null,"""entra_devengado""",-0.0,100.0,1100.0,-0.0,1.0,1.0,1.0,0.0,0.0,null,null,null,null,null
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""recibo""",-1,0,2024-01-30,null,null,null,null,null,null,null,null,null,null,null,null,null,…,"""ifrs4""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-31,2024-01-30,2024-01-30,2024-02-15,50.0,0,0,true,0,0,0.0,null,null,null,null,"""finalizado""",-0.0,0.0,0.0,-0.0,1.0,1.0,1.0,0.0,0.0,null,null,null,null,null
"""modificacion_valorable_autos""","""produccion_directo""","""directo""",323423423,2024-01-30,2938433191,"""DANOS""",323423843,"""01""","""040""","""sucursal""","""plan_basico""","""modificacion_valorable""","""COP""",2024-01-30,2024-01-30,2024-02-15,2024-01-30,2024-02-15,50.0,"""cobertura""",-1,0,2024-01-30,null,null,null,null,null,null,null,null,null,null,null,null,null,…,"""ifrs17_corporativo""","""diario""","""directo""",2024,"""no_aplica""",0,2024-12-01,2024-12-3

In [16]:
# solo los casos en que aplica deterioro
output_deterioro.filter(pl.col('liberacion_deterioro').is_not_null())

aux_casuistica,tipo_insumo,tipo_negocio,poliza,fecha_expedicion_poliza,recibo,amparo,poliza_certificado,compania,ramo_sura,canal,producto,tipo_op,moneda,fecha_contabilizacion_recibo,fecha_inicio_vigencia_recibo,fecha_fin_vigencia_recibo,fecha_inicio_vigencia_cobertura,fecha_fin_vigencia_cobertura,valor_prima_emitida,nivel_detalle,signo_constitucion,candidato_devengo_50_50,fecha_inicio_vigencia,tipo_gasto,porc_gasto,prioridad_match,rn,podto_comercial,podto_tecnico,contrato_reaseguro,nit_reasegurador,tipo_reasegurador,porc_participacion_reasegurador,fe_ini_vig_contrato_reaseguro,fe_fin_vig_contrato_reaseguro,valor_prima_cedida,…,tipo_contabilidad,regla_devengo,tipo_contrato,cohorte,anio_liberacion,transicion,fecha_inicio_periodo,fecha_valoracion,fecha_constitucion,fecha_inicio_devengo,fecha_fin_devengo,valor_base_devengo,dias_devengados,dias_no_devengados,control_suma_dias,dias_constitucion,dias_liberacion,valor_devengo_diario,mes_constitucion,mes_ini_liberacion,mes_fin_liberacion,porc_consumo_limite,estado_devengo,valor_constitucion,valor_liberacion,valor_liberacion_acum,saldo,tasa_cambio_fecha_valoracion,tasa_cambio_fecha_valoracion_anterior,tasa_cambio_fecha_constitucion,fluctuacion_constitucion,fluctuacion_liberacion,prob_incumplimiento_actual,prob_incumplimiento_anterior,cambio_prob_incumplimiento,constitucion_deterioro,liberacion_deterioro
str,str,str,i64,date,i64,str,i64,str,str,str,str,str,str,date,date,date,date,date,f64,str,i64,i32,date,str,f64,i32,i64,f64,f64,i64,i64,str,f64,date,date,i64,…,str,str,str,i32,str,i32,date,date,date,date,date,f64,i64,i64,bool,i64,i64,f64,i32,i32,i32,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""mensual_mes_fin""","""comision_rea_prop""","""mantenido""",345675387,null,5260869017,"""HURTO""",345675807,"""01""","""040""",null,"""plan_2""",null,"""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,null,"""cobertura""",-1,0,2024-01-15,null,null,null,null,null,null,1245,3693358,"""ext""",0.5,null,null,null,…,"""ifrs17_corporativo""","""diario""","""reaseguro_proporcional""",null,"""no_aplica""",0,2024-12-01,2024-12-31,2024-02-20,2024-02-20,2024-02-15,3.0,352,0,false,32,32,0.09375,null,null,null,null,"""entra_devengado""",-0.0,3.0,33.0,-0.0,1.0,1.0,1.0,0.0,0.0,0.05,0.07,-0.02,-0.0,0.0
"""mensual_mes_fin""","""comision_rea_prop""","""mantenido""",345675387,null,5260869017,"""HURTO""",345675807,"""01""","""040""",null,"""plan_2""",null,"""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,null,"""cobertura""",-1,0,2024-01-15,null,null,null,null,null,null,1245,3693358,"""ext""",0.5,null,null,null,…,"""ifrs17_local""","""diario""","""reaseguro_proporcional""",null,"""no_aplica""",0,2024-12-01,2024-12-31,2024-02-20,2024-02-20,2024-02-15,3.0,352,0,false,32,32,0.09375,null,null,null,null,"""entra_devengado""",-0.0,3.0,33.0,-0.0,1.0,1.0,1.0,0.0,0.0,0.05,0.07,-0.02,-0.0,0.0
"""mensual_mes_fin""","""comision_rea_prop""","""mantenido""",345675387,null,5260869017,"""HURTO""",345675807,"""01""","""040""",null,"""plan_2""",null,"""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,null,"""recibo""",-1,0,2024-01-15,null,null,null,null,null,null,1245,3693358,"""ext""",0.5,null,null,null,…,"""ifrs4""","""diario""","""reaseguro_proporcional""",null,"""no_aplica""",0,2024-12-01,2024-12-31,2024-02-20,2024-02-20,2024-02-15,3.0,352,0,false,32,32,0.09375,null,null,null,null,"""entra_devengado""",-0.0,3.0,33.0,-0.0,1.0,1.0,1.0,0.0,0.0,0.05,0.07,-0.02,-0.0,0.0
"""mensual_mes_fin""","""comision_rea_prop""","""mantenido""",345675387,null,1292794322,"""HURTO""",345675807,"""01""","""040""",null,"""plan_2""",null,"""COP""",2024-02-20,2024-01-15,2024-02-15,2024-01-15,2024-02-15,null,"""cobertura""",-1,0,2024-01-15,null,null,null,null,null,null,1245,2374188,"""int""",0.5,null,null,null,…,"""ifrs17_corporativo""","""diario""","""reaseguro_proporcional""",null,"""no_aplica""",0,2024-12-01,2024-12-31,2024-02-20,2024-02-20,2024-02-15,3.0,352,0,false,32,32,0.09375,null,null,nul

# 5. Generación del Output Contable
- Cada output debe mapearse a una Business Transaction BT con su descripción
- [Pendiente en otra HU] Además del output actuarial, deben incluirse los valores de cuentas por cobrar del seguro directo y la apertura de la cuenta corriente del reaseguro con sus respectivas BTs
- Cada output debe tener los campos descriptivos con sus códigos de la sábana contable (insumo transversal)

In [18]:
# Esta función implementa el mapeo a bts contables, además de asignar los campos de la sábana
#output_contable = mapcont.gen_output_contable(output_fluct, input_map_bts, input_tipo_seguro)
#output_contable.write_excel('./output/output_contable_muestra.xlsx')
#output_contable